<a href="https://colab.research.google.com/github/totminaekaterina/NLP/blob/main/Yargy_parser_for_extracting_facts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
!pip install ipymarkup
!pip install yargy

In [215]:
from yargy import Parser, rule, and_, or_

In [216]:
from yargy.predicates import (
    eq, in_, dictionary,
    type, gram, normalized)

In [217]:
from yargy.interpretation import fact, attribute

In [218]:
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, RED, GREEN
from ipymarkup import show_span_ascii_markup as show_markup

In [219]:
from yargy.pipelines import morph_pipeline

In [220]:
from IPython.display import display

#Data

In [221]:
! gdown --id 1NXceWoUR20iL1k0RYvYJwTp1fOyyIVig

Downloading...
From: https://drive.google.com/uc?id=1NXceWoUR20iL1k0RYvYJwTp1fOyyIVig
To: /content/texts.txt
100% 4.24k/4.24k [00:00<00:00, 4.93MB/s]


In [222]:
from random import seed, sample


def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')

            
def make_translation_table(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation_table(
    '‑–—−',
    '----'
)


def normalize_text(text):
    return text.translate(DASHES_TRANSLATION)


texts = [
    normalize_text(_)
    for _ in load_lines('texts.txt')
]

In [223]:
import re
import json

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    rule, or_,
    not_, and_
)
from yargy.predicates import (
    eq, type, caseless, in_, in_caseless,
    gte, lte, length_eq,
    is_capitalized, normalized,
    dictionary, gram,
)
from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp
from yargy.relations import gnc_relation
from yargy.tokenizer import (
    QUOTES,
    LEFT_QUOTES,
    RIGHT_QUOTES,

    MorphTokenizer,
    TokenRule
)

In [224]:
def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def test_interpretation(rule, *tests):
    parser = Parser(rule)
    for line, etalon in tests:
        match = parser.match(line)
        assert match, line
        guess = match.fact
        assert etalon == guess, guess
        

def test_match(rule, *tests):
    parser = Parser(rule)
    for line in tests:
        match = parser.match(line)
        assert match, line

#COMMON

In [225]:
INT = type('INT')
LATIN = type('LATIN')
DOT = eq('.')
DASH = eq('-')

#MAIN TYPE

In [226]:
MAIN_TYPE = morph_pipeline([
    'Название',
    'Текст',
    'Речь'
]).interpretation(
    interp.normalized()
)

#ORGANISATION

In [227]:
QUOTE = in_(QUOTES)
LEFT_QUOTE = in_(LEFT_QUOTES)
RIGHT_QUOTE = in_(RIGHT_QUOTES)

ORGANISATION = or_(
    rule(
        LEFT_QUOTE,
        not_(RIGHT_QUOTE).repeatable(),
        RIGHT_QUOTE,
    ),
    rule(
        and_(
            QUOTE,
            not_(RIGHT_QUOTE)
        ),
        not_(QUOTE).repeatable(),
        and_(
            QUOTE,
            not_(LEFT_QUOTE)
        )
    )
)

test_match(
    ORGANISATION,
    '"Газпромнефть - НГУ"',
    '"Газпром нефти"',
    '"GeoHack"',
    '"РНФ"'
)

#EVENT

In [228]:
Event = fact(
    'Event',
    ['type']
)


TYPE = morph_pipeline([
    'магистерская программа',
    'зимняя школа',
    'региональный конкурс грантов',
    'НОЦ'
]).interpretation(
    Event.type.normalized()
)


EVENT = rule(
    TYPE.optional(),
).interpretation(
    Event
)


test_interpretation(
    EVENT,
    ['магистерские программы', Event(type='магистерская программа')],
    ['зимняя школа', Event(type='зимняя школа')],
    ['региональном конкурсе грантов', Event(type='региональный конкурс грантов')],
    ['НОЦ', Event(type='НОЦ')]
)

#NAME

In [229]:
Name = fact(
    'Name',
    ['first', 'middle', 'last']
)


gnc = gnc_relation()

LAST = and_(
    type('RU'),
    is_capitalized()
).interpretation(
    Name.last.inflected()
).match(gnc)

FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

MIDDLE = gram('Patr').interpretation(
    Name.middle.inflected()
).match(gnc)

ABBR = and_(
    length_eq(1),
    is_capitalized()
)

FIRST_ABBR = ABBR.interpretation(
    Name.first.custom(str.lower)
)

MIDDLE_ABBR = ABBR.interpretation(
    Name.middle.custom(str.lower)
)


NAME = or_(
    rule(
        LAST,
        FIRST_ABBR, DOT,
        MIDDLE_ABBR, DOT
    ),
    rule(
        FIRST_ABBR, DOT,
        MIDDLE_ABBR, DOT,
        LAST
    ),
    rule(
        LAST,
        FIRST_ABBR, DOT
    ),
    rule(
        FIRST_ABBR, DOT,
        LAST
    ),
    rule(
        FIRST,
        MIDDLE,
        LAST
    ),
    rule(
        LAST,
        FIRST,
        MIDDLE
    ),
    rule(
        FIRST,
        LAST
    ),
).interpretation(
    Name
)


test_interpretation(
    NAME,
    ['Алексей Вашкевич', Name(first='алексей', last='вашкевич')],
    ['Дмитрия Новикова', Name(first='дмитрий', last='новиков')]
)

#PERSON

In [230]:
Person = fact(
    'Person',
    ['type', 'name']
)


TYPE = morph_pipeline([
    'ведущий научный сотрудник',
    'директор по технологическому развитию'
]).interpretation(
    Person.type.normalized()
)

PERSON = or_(
    NAME,
    TYPE
).interpretation(
    Person.name
)

PERSON = rule(
    TYPE.optional(),
    PERSON
).interpretation(
    Person
)


test_interpretation(
    PERSON,
    [ 'директор по технологическому развитию Алексей Вашкевич',
        Person(type='директор по технологическому развитию', name=Name(first='алексей', last='вашкевич'))
    ],
    ['ведущего научного сотрудника Дмитрия Новикова',
        Person(type='ведущий научный сотрудник', name=Name(first='дмитрий', last='новиков'))
    ]
)

#DATE

In [231]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12,
}

MONTH_NAME = dictionary(MONTHS).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

MONTH = and_(
    INT,
    gte(1),
    lte(12)
).interpretation(
    Date.month.custom(int)
)

YEAR = and_(
    INT,
    gte(1000),
    lte(3000)
).interpretation(
    Date.year.custom(int)
)

YEAR_SUFFIX = rule(
    or_(
        eq('г'),
        normalized('году')
    ),
    DOT.optional()
)

DAY = and_(
    INT,
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

DATE = or_(
    rule(
        YEAR,
        YEAR_SUFFIX
    ),
    rule(
        MONTH_NAME,
        YEAR
    ),
    rule(
        DAY,
        DOT,
        MONTH,
        DOT,
        YEAR
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR
    ),
    rule(
        DAY,
        MONTH_NAME
    )

)

SUFFIX = normalized('началось')

DATE = rule(
    DATE,
    SUFFIX.optional()
).interpretation(
    Date
)

test_interpretation(
    DATE,
    ['2018 году', Date(year=2018)],
    ['2021 года', Date(year=2021)],
    ['31 января', Date(month=1, day=31)],
    ['6 февраля ', Date(month=2, day=6)]
)

#TASK

In [232]:
Task = fact(
    'Task',
    ['action', 'object']
)

ACTION = morph_pipeline([
    'создание',
    'развитие',
    'картирование',
    'обработка',
    'физическое и математическое моделирование',
]).interpretation(
    Task.action.normalized()
)

OBJECT = morph_pipeline([
    'технологии геологического хранения углеводорода',
    'портфель проектов в области CCUS',
    'объекты захоронения СО2',
    'инструменты контроля',
    'сейсморазведочных данных',
    'гидроразрыв пласта',
    'цифровые двойники горных пород',
]).interpretation(
    Task.object.normalized()
)


TASK = rule(
    ACTION,
    OBJECT
).interpretation(
    Task
)


test_interpretation(
    TASK,
    ['созданию технологии геологического хранения углеводорода', Task(action='создание', object='технологии геологического хранения углеводорода')],
    #['развивает портфель проектов в области CCUS', Task(action='развитие', object='портфель проектов в области CCUS')],
    #['картированию объектов захоронения СО2 ', Task(action='картирование', object='объекты захоронения СО2')],
    ['развитию инструментов контроля', Task(action='развитие', object='инструменты контроля')],
    ['обработки сейсморазведочных данных', Task(action='обработка', object='сейсморазведочных данных')],
    ['физического и математического моделирования гидроразрыва пласта', Task(action='физическое и математическое моделирование', object='гидроразрыв пласта')],
    ['создания цифровых двойников горной породы', Task(action='создание', object='цифровые двойники горных пород')]
)

#FSEM

In [238]:
Item = fact(
    'Item',
    ['type',
     'organisation',
     'person',
     'date',
     'task']
)

Fsem = fact(
    'Fsem',
    [attribute('items').repeatable(), 'event']
)


ATTRIBUTE = or_(
    ORGANISATION.interpretation(Item.organisation),
    PERSON.interpretation(Item.person),
    TASK.interpretation(Item.task),
    DATE.interpretation(Item.date)
)

ITEM = rule(
    MAIN_TYPE.interpretation(Item.type),
    ATTRIBUTE.repeatable()
).interpretation(
    Item
)

FSEM = rule(
    ITEM.interpretation(
        Fsem.items
    ).repeatable(),
    EVENT.interpretation(
        Fsem.event
    )
).interpretation(
    Fsem
)

#EXTRACTOR

In [239]:
def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


class Match(object):
    def __init__(self, fact, spans):
        self.fact = fact
        self.spans = spans


DEBUG = or_(
    MAIN_TYPE,
    ORGANISATION,
    PERSON,
    DATE,
    TASK
)

TOKENIZER = MorphTokenizer()

class Extractor(object):
    def __init__(self):
        self.debug = Parser(DEBUG, tokenizer=TOKENIZER)
        self.parser = Parser(FSEM, tokenizer=TOKENIZER)

    def __call__(self, text):
        matches = self.debug.findall(text)
        spans = [_.span for _ in matches]

        line = join_spans(text, spans)
        matches = list(self.parser.findall(line))
        fact = None
        if matches:
            match = matches[0]
            fact = match.fact

        return Match(fact, spans)

    
extractor = Extractor()

seed()
for text in texts:
    match = extractor(text)
    show_markup(text, match.spans)
    if match.fact:
        show_json(match.fact.as_json)

Название. НОЦ «Газпромнефть-НГУ» получил награду за вклад в развитие 
────────      ──────────────────                                     
инновационной экосистемы.
{
  "items": [
    {
      "type": "Название",
      "organisation": "«Газпромнефть-НГУ»"
    }
  ]
}
Текст. Научно-образовательный центр «Газпромнефть-НГУ» получил награду
─────                               ──────────────────                
 от Научно-Технического «Центра Газпром нефти» за значительный вклад в
                        ──────────────────────                        
 развитие инновационной экосистемы компании.
{
  "items": [
    {
      "type": "Текст",
      "organisation": "«Центра Газпром нефти»"
    }
  ]
}
Текст. Таким образом индустриальный партнер Новосибирского 
─────                                                      
государственного университета выразил благодарность за выстраивание 
стратегии взаимодействия, создание и настройку операционной модели 
проектного офиса по взаимодействию с компан